In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models
import zipfile

# Step 1: Specify the Path to the Uploaded Zip File
zip_path = "/content/archieve.zip"  # Use the exact filename of the uploaded zip

# Step 2: Extract the Zip File
# Extract the zip file to a folder named "HandGestures"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("HandGestures")

# Step 3: Set the Data Directory Path After Extraction
# Update this based on the extracted folder structure
data_dir = "HandGestures/leapGestRecog"  # This should point to the main directory after extraction

# Check the extracted folder structure to confirm
print("Directory Structure after Extraction:")
print(os.listdir(data_dir))

# Step 4: Load and Preprocess Images from Nested Folders
def load_images_from_directory(root_dir, target_size=(227, 227)):
    """
    Traverse nested directories to load images and labels.
    Each class label is derived from the folder name (e.g., '01_palm' -> class label).
    """
    images, labels = [], []

    # Traverse through the main directory and its subdirectories
    for main_dir in os.listdir(root_dir):
        main_dir_path = os.path.join(root_dir, main_dir)  # e.g., "HandGestures/leapGestRecog/01"
        if not os.path.isdir(main_dir_path):
            continue

        # Traverse each gesture folder inside each main directory (e.g., "HandGestures/leapGestRecog/01/01_palm")
        for gesture_dir in os.listdir(main_dir_path):
            gesture_dir_path = os.path.join(main_dir_path, gesture_dir)  # e.g., "HandGestures/leapGestRecog/01/01_palm"

            if not os.path.isdir(gesture_dir_path):
                continue

            # The gesture label can be derived from the folder name
            label = gesture_dir  # Use folder names as class labels (e.g., "01_palm")

            # Traverse through each image in the gesture directory
            for img_name in os.listdir(gesture_dir_path):
                img_path = os.path.join(gesture_dir_path, img_name)
                try:
                    # Load the image, resize it to the target size, and convert to numpy array
                    img = load_img(img_path, target_size=target_size)
                    img_array = img_to_array(img)
                    images.append(img_array)
                    labels.append(label)  # Use folder name as label
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

    # Convert lists to numpy arrays
    return np.array(images, dtype='float32'), np.array(labels)

# Load images and labels from the nested folder structure
images, labels = load_images_from_directory(data_dir)

# Convert class labels to numerical values using a label encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)  # Convert string labels to integers

# Normalize the images to the range [0, 1]
images = images / 255.0

# Step 5: Split the Dataset into Training and Testing Sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.3, random_state=42
)

# Step 6: Define the AlexNet Model
def build_alexnet(input_shape=(227, 227, 3), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((3, 3), strides=2),
        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.MaxPooling2D((3, 3), strides=2),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((3, 3), strides=2),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Output layer for classification
    ])
    return model

# Step 7: Build and Compile the Model
input_shape = (227, 227, 3)
num_classes = len(np.unique(labels))  # Number of unique class labels
alexnet_model = build_alexnet(input_shape=input_shape, num_classes=num_classes)
alexnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 8: Train the Model
history = alexnet_model.fit(
    train_images, train_labels,  # Training data
    validation_data=(test_images, test_labels),  # Validation data
    epochs=15,  # Adjust the number of epochs based on performance
    batch_size=32,  # Adjust batch size if needed
    verbose=2  # Print progress during training
)

# Step 9: Evaluate the Model
test_loss, test_acc = alexnet_model.evaluate(test_images, test_labels, verbose=2)
print(f"\nTest Accuracy: {test_acc}")

# Step 10: Visualize the Training and Validation Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy over Epochs')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/archieve.zip'